<a href="https://colab.research.google.com/github/kameda-yoshinari/IMISToolExeA2021/blob/main/600/pytorch_advanced-revised/4_pose_estimation/GC4_7_OpenPose_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 4.7 推論の実施

- 本ファイルでは、学習させたOpenPoseで姿勢推定を行います。


# 学習目標


1.	OpenPoseの学習済みモデルをロードできるようになる
2.	OpenPoseの推論を実装できるようになる



---

# Google Colab

In [ ]:
!echo "Change to the JST notation."
!rm /etc/localtime
!ln -s /usr/share/zoneinfo/Japan /etc/localtime

Change to the JST notation.


In [ ]:
!echo "Start mounting your Google Drive."
from google.colab import drive 
drive.mount('/content/drive')
%cd /content/drive/My\ Drive/
!echo "Move to the working directory."
%cd 202107_Tool-A/Work600/
!ls -l

Start mounting your Google Drive.
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive
Move to the working directory.
/content/drive/My Drive/202107_Tool-A/Work600
total 4
drwx------ 13 root root 4096 Jul 31 09:41 pytorch_advanced


---
# 共通準備

"pytorch_advanced" folder should be ready before you come here.

In [ ]:
# Skip this if you have already issued git in advance. 
# If you come here by way of 600-PyTorchADL.ipynb, 
# you should skip the git command (as you have already issued in 600).  
# If you run git when pytorch_advanced already exists, git tells the error and clone won't be made.

#!git clone https://github.com/YutaroOgawa/pytorch_advanced.git

import os
if os.path.exists("/content/drive/My Drive/202107_Tool-A/Work600/pytorch_advanced"):
    print("OK. Alreadly git cloned. You can go.")
else:
    print("You'd better go back to the first 600-PyTorchADL.ipynb")

OK. Alreadly git cloned. You can go.


In [ ]:
!ls

pytorch_advanced


In [ ]:
%cd "pytorch_advanced"

/content/drive/My Drive/202107_Tool-A/Work600/pytorch_advanced


In [ ]:
!ls

1_image_classification	 7_nlp_sentiment_transformer
2_objectdetection	 8_nlp_sentiment_bert
3_semantic_segmentation  9_video_classification_eco
4_pose_estimation	 etc
5_gan_generation	 LICENSE
6_gan_anomaly_detection  README.md


In [ ]:
%cd "4_pose_estimation"

/content/drive/My Drive/202107_Tool-A/Work600/pytorch_advanced/4_pose_estimation


In [ ]:
!ls

4-2_DataLoader.ipynb	     4-7_OpenPose_inference.ipynb	    utils
4-3-4_NetworkModel.ipynb     data				    weights
4-5_TensorBoardX.ipynb	     make_folders_and_data_downloads.ipynb
4-6_OpenPose_training.ipynb  tbX


---
# Utils, Data, and Weights

Working directories are:

* /root/utils  
* /root/data  
* /root/weights  

val2014.zip and mask.tar.gz are expanded there.

In [ ]:
!ls -l '/content/drive/My Drive/202107_Tool-A/Work600/pytorch_advanced/4_pose_estimation/'

total 2628
-rw------- 1 root root 1273983 Jul 31 09:42 4-2_DataLoader.ipynb
-rw------- 1 root root  150239 Jul 31 09:42 4-3-4_NetworkModel.ipynb
-rw------- 1 root root   15393 Jul 31 09:42 4-5_TensorBoardX.ipynb
-rw------- 1 root root   14793 Jul 31 09:42 4-6_OpenPose_training.ipynb
-rw------- 1 root root 1213399 Jul 31 09:42 4-7_OpenPose_inference.ipynb
drwx------ 2 root root    4096 Jul 31 09:42 data
-rw------- 1 root root    5310 Jul 31 09:42 make_folders_and_data_downloads.ipynb
drwx------ 2 root root    4096 Aug  1 14:51 tbX
drwx------ 3 root root    4096 Jul 31 09:42 utils
drwx------ 2 root root    4096 Aug  1 12:49 weights


In [ ]:
# This will take 8 minutes.

%cd ~

!cp -r '/content/drive/My Drive/202107_Tool-A/Work600/pytorch_advanced/4_pose_estimation/utils/'   .
!cp -r '/content/drive/My Drive/202107_Tool-A/Work600/pytorch_advanced/4_pose_estimation/data/'    .
!cp -r '/content/drive/My Drive/202107_Tool-A/Work600/pytorch_advanced/4_pose_estimation/weights/' .

%cd data
!unzip -q -o  val2014.zip
!tar xfz mask.tar.gz

%cd ~

/root


In [ ]:
!ls -l utils data weights

---
# 事前準備

- 学習済みの重みパラメータ「pose_model_scratch.pth」をフォルダ「weights」に用意済のはずです。
- ※ Issue [#142] (https://github.com/YutaroOgawa/pytorch_advanced/issues/142) 対策済



In [ ]:
from PIL import Image
import cv2
import numpy as np
from matplotlib import cm
import matplotlib.pyplot as plt
%matplotlib inline

import torch

In [ ]:
from utils.openpose_net import OpenPoseNet

# 学習済みモデルと本章のモデルでネットワークの層の名前が違うので、対応させてロードする
# モデルの定義
net = OpenPoseNet()

# 学習済みパラメータをロードする
net_weights = torch.load(
    './weights/pose_model_scratch.pth', map_location={'cuda:0': 'cpu'})
keys = list(net_weights.keys())

weights_load = {}

# ロードした内容を、本書で構築したモデルの
# パラメータ名net.state_dict().keys()にコピーする
for i in range(len(keys)):
    weights_load[list(net.state_dict().keys())[i]
                 ] = net_weights[list(keys)[i]]

# コピーした内容をモデルに与える
state = net.state_dict()
state.update(weights_load)
net.load_state_dict(state)

print('ネットワーク設定完了：学習済みの重みをロードしました')


In [ ]:
# 草野球の画像を読み込み、前処理します

test_image = './data/hit-1407826_640.jpg'
oriImg = cv2.imread(test_image)  # B,G,Rの順番

# BGRをRGBにして表示
oriImg = cv2.cvtColor(oriImg, cv2.COLOR_BGR2RGB)
plt.imshow(oriImg)
plt.show()

# 画像のリサイズ
size = (368, 368)
img = cv2.resize(oriImg, size, interpolation=cv2.INTER_CUBIC)

# 画像の前処理
img = img.astype(np.float32) / 255.

# 色情報の標準化
color_mean = [0.485, 0.456, 0.406]
color_std = [0.229, 0.224, 0.225]

# 21/03/07 Issue147 https://github.com/YutaroOgawa/pytorch_advanced/issues/147
# 色チャネルの順番を誤っています
# preprocessed_img = img.copy()[:, :, ::-1]  # RGB→BGR
preprocessed_img = img.copy()  # RGB

for i in range(3):
    preprocessed_img[:, :, i] = preprocessed_img[:, :, i] - color_mean[i]
    preprocessed_img[:, :, i] = preprocessed_img[:, :, i] / color_std[i]

# （高さ、幅、色）→（色、高さ、幅）
img = preprocessed_img.transpose((2, 0, 1)).astype(np.float32)

# 画像をTensorに
img = torch.from_numpy(img)

# ミニバッチ化：torch.Size([1, 3, 368, 368])
x = img.unsqueeze(0)


In [ ]:
# OpenPoseでheatmapsとPAFsを求めます
net.eval()
predicted_outputs, _ = net(x)

# 画像をテンソルからNumPyに変化し、サイズを戻します
pafs = predicted_outputs[0][0].detach().numpy().transpose(1, 2, 0)
heatmaps = predicted_outputs[1][0].detach().numpy().transpose(1, 2, 0)

pafs = cv2.resize(pafs, size, interpolation=cv2.INTER_CUBIC)
heatmaps = cv2.resize(heatmaps, size, interpolation=cv2.INTER_CUBIC)

pafs = cv2.resize(
    pafs, (oriImg.shape[1], oriImg.shape[0]), interpolation=cv2.INTER_CUBIC)
heatmaps = cv2.resize(
    heatmaps, (oriImg.shape[1], oriImg.shape[0]), interpolation=cv2.INTER_CUBIC)


In [ ]:
# 左肘と左手首のheatmap、そして左肘と左手首をつなぐPAFのxベクトルを可視化する
# 左肘
heat_map = heatmaps[:, :, 6]  # 6は左肘
heat_map = Image.fromarray(np.uint8(cm.jet(heat_map.numpy())*255))
heat_map = np.asarray(heat_map.convert('RGB'))

# 合成して表示
blend_img = cv2.addWeighted(oriImg, 0.5, heat_map, 0.5, 0)
plt.imshow(blend_img)
plt.show()


# 左手首
heat_map = heatmaps[:, :, 7]  # 7は左手首
heat_map = Image.fromarray(np.uint8(cm.jet(heat_map.numpy())*255))
heat_map = np.asarray(heat_map.convert('RGB'))

# 合成して表示
blend_img = cv2.addWeighted(oriImg, 0.5, heat_map, 0.5, 0)
plt.imshow(blend_img)
plt.show()


# 左肘と左手首をつなぐPAFのxベクトル
paf = pafs[:, :, 24]
paf = Image.fromarray(np.uint8(cm.jet(paf)*255))
paf = np.asarray(paf.convert('RGB'))

# 合成して表示
blend_img = cv2.addWeighted(oriImg, 0.5, paf, 0.5, 0)
plt.imshow(blend_img)
plt.show()


In [ ]:
from utils.decode_pose import decode_pose
_, result_img, _, _ = decode_pose(oriImg, heatmaps, pafs)


In [ ]:
# 結果を描画
plt.imshow(oriImg)
plt.show()

plt.imshow(result_img)
plt.show()


以上

---
Revised by KAMEDA, Yoshinari at University of Tsukuba for lecture purpose.  
Original: https://github.com/YutaroOgawa/pytorch_advanced

2021/08/02. 